In [1]:
!pip install -q pypdf4

In [9]:
import PyPDF4

In [46]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_4c6fa590fc764326b69305ee65fa07d2 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='BCl6gltfE5Luk5slOpcu9Nlrq9hxQrn4m0H5z8Gm2q1f',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_4c6fa590fc764326b69305ee65fa07d2.get_object(Bucket='covid-donotdelete-pr-zhbc5zrjtrt3nt',Key='time_series-ncov-Confirmed.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data = pd.read_csv(body)
df_data.head()


,Province/State,Country/Region,Lat,Long,Date,Value
0,#adm1+name,#country+name,#geo+lat,#geo+lon,#date,#affected+infected+value+num
1,NaN,Afghanistan,33,65,2020-03-16,21
2,NaN,Afghanistan,33,65,2020-03-15,16
3,NaN,Afghanistan,33,65,2020-03-14,11
4,NaN,Afghanistan,33,65,2020-03-13,7


In [49]:
import numpy as np

In [59]:
df_data.head()

,Province/State,Country/Region,Lat,Long,Date,Value
0,#adm1+name,#country+name,#geo+lat,#geo+lon,#date,#affected+infected+value+num
1,NaN,Afghanistan,33,65,2020-03-16,21
2,NaN,Afghanistan,33,65,2020-03-15,16
3,NaN,Afghanistan,33,65,2020-03-14,11
4,NaN,Afghanistan,33,65,2020-03-13,7


In [60]:
new_df = df_data.drop(["Province/State","Lat","Long","Date"],axis=1)

In [61]:
new_df.head()

,Country/Region,Value
0,#country+name,#affected+infected+value+num
1,Afghanistan,21
2,Afghanistan,16
3,Afghanistan,11
4,Afghanistan,7


In [62]:
new_df = new_df.drop([0],axis=0)

In [135]:
names = new_df["Country/Region"].unique()

In [63]:
new_df.head(2)

,Country/Region,Value
1,Afghanistan,21
2,Afghanistan,16


In [158]:
def sum_column(frame,country):
    new = frame[frame["Country/Region"] == country]
    return sum(new["Value"].apply(int))

In [159]:
sums = []
for name in names:
    sums.append(sum_column(new_df,name))

In [92]:

body = client_4c6fa590fc764326b69305ee65fa07d2.get_object(Bucket='covid-donotdelete-pr-zhbc5zrjtrt3nt',Key='time_series-ncov-Deaths.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_1 = pd.read_csv(body)
df_data_1.head()


,Province/State,Country/Region,Lat,Long,Date,Value
0,#adm1+name,#country+name,#geo+lat,#geo+lon,#date,#affected+killed+value+num
1,NaN,Afghanistan,33,65,2020-03-16,0
2,NaN,Afghanistan,33,65,2020-03-15,0
3,NaN,Afghanistan,33,65,2020-03-14,0
4,NaN,Afghanistan,33,65,2020-03-13,0


In [93]:
df_new = df_data_1.drop(["Province/State","Lat","Long","Date"],axis=1)

In [95]:
df_new = df_new.drop([0],axis=0)

In [140]:
df_new.head()

,Country/Region,Value
1,Afghanistan,0
2,Afghanistan,0
3,Afghanistan,0
4,Afghanistan,0
5,Afghanistan,0


In [160]:
death_sums = []
for name in names:
    death_sums.append(sum_column(df_new,name))

In [166]:
final = np.array((sums,death_sums))
final

array([[     95,     211,     340,      16,       4,     258,     119,
              8,    2504,    4278,     152,    2071,      34,     188,
           4926,       1,      11,      38,     116,     875,     205,
            193,      27,      79,      22,    2003,       2,     422,
        2962462,     148,       2,      10,     171,       6,     336,
          21260,      19,     116,    1237,    4990,      72,     256,
            796,       2,     666,       3,       8,    1215,   31338,
             11,       3,     240,   32263,      15,    1677,       1,
             11,       3,       4,       3,       4,      11,      11,
             17,     181,    1181,     911,     552,  117303,    1059,
            730,    1473,  170830,      39,   10817,       6,      37,
             29,       8,  121156,       4,    1329,     149,     730,
              1,      22,      61,     268,    3009,      75,     107,
             50,       3,       1,     215,      75,      27,       7,
      

In [169]:
final = np.array(list(zip(sums,death_sums)))

In [171]:
final.shape

(156, 2)

In [176]:
result = pd.DataFrame(data=final,index=names,columns=["Confirmed","Death"])

In [178]:
result.to_csv("covid-19.csv")